Programming assignment 2

Consider a set of pictures from 360° camera mounted inside a merry-go-round. They were taken at night and only one seat is visible which emits light – the seat in the shape of a jelly fish. We know that the merry-go-round rotates clockwise and that all the pictures were taken during a single cycle of merry-go-round. Given that the first picture is P1, your task is to sort the rest in the chronological order. Assume that the move of jelly fish is monotonous in horizontal direction, i.e., it always moves in clockwise direction, never backwards.

Input files: text files, each of them with 10 rows and 80 columns representing the brightness level in the given parts of the picture. Jelly fish can be recognized by high brightness (value 1 to 9) on a black background (value 0).

00000000000000000000000000000000000000000000000000000000000000000000000000000000
00000000000000000000000000000000000000000000000000000000000000000000000000000000
00000000000000000000000000000000000000000000000000000000000000000000000000000000
00000000000000000000000000000000000000000000000000000000000000000000000000000000
00000000000000000000000000000000000000000000000000000000000000000000000000000000
00000000000000000000000000000000000005950000000000000000000000000000000000000000
00000000000000000000000000000000000009990000000000000000000000000000000000000000
00000000000000000000000000000000000005950000000000000000000000000000000000000000
00000000000000000000000000000000000003930000000000000000000000000000000000000000
00000000000000000000000000000000000000000000000000000000000000000000000000000000
Directions

In order to capture the movement of the jelly fish between two pictures, use Earth-mover distance, also known as Wasserstein distance. It can be formulated as a min-cost flow problem on a certain network. Use NetworkX package (https://networkx.org) to perform the optimization required to find this flow.

About Earth-mover distance: Consider two distributions over a discrete set of points P: μ in [0,1]^P and λ in [0,1]^P. Since both are distributions and sum of μ(p) over all points in P is 1 (and the same holds for λ), there is surely a way to transform μ into λ: we denote fp,q the probability mass transfered from point p to q for each pair of points p,q from P, in order to transform μ into λ:

For each p in P: μ(p) = Σq fp,q and for each q in P: λ(q) = Σp fp,q. 

Note that fp,p denotes the probability mass which remains at the same place.
Let d(p,q) denote the distance between p and q. Then, the cost of the transformation f is

Σp,q d(p,q) fp,q

The earth-mover distance of μ and λ is then the cost of the cheapest transformation between μ and λ.

Technical instructions:

Your code should implement function sort_files() which returns ordered list of the pictures, something like this:

['P1.txt', 'P2.txt', 'P3.txt', 'P10.txt', 'P4.txt', 'P13.txt', 'P14.txt', 'P9.txt',
  'P15.txt', 'P7.txt', 'P12.txt', 'P11.txt', 'P5.txt', 'P6.txt', 'P8.txt']
You should also define and use function comp_dist(file1, file2) which returns the Earth-mover distance between file1 and file2. I may want to access it during the grading process, especially if your output is not completely correct.

Make sure your code never breaks down. If you know that it cannot handle e.g. P10.txt, skip this file and don't include it in the output. This way, you will not get a full score, but you will still get points for the partial solution.

Before submission, always try your code in vocareum using "Run" button. Incorrect output format, syntax and indentation errors, etc will result into 0 points.

Firstly we define a function which given two files calculated the EMD distance between them. We create a graph with a sink node 's' and a tank node 't' with an additional 'jolly edge' which solves some rounding problems which will would have emerged later.

In [ ]:
import numpy as np
import networkx as nx

# This function returns the EMD distances between file1 and file2.

def comp_dist(file1, file2):
    with open(file1, 'r') as f1, open(file2, 'r') as f2:
        f_1 = []
        f_2 = []
        for lines in f1:
            f_1.append(lines[:-1])
        for lines in f2:
            f_2.append(lines[:-1])             
        all_lists = [f_1,f_2]
        G = nx.DiGraph()
        s1 = 0 
        for x in f_1:
            for i in range(len(x)):
                s1+= int(x[i])
        s2=0
        for x in f_2:
            for i in range(len(x)):
                s2+= int(x[i])
        G.add_node('s', demand= -1)
        G.add_node('t', demand= 1)
        G.add_edge('s', 't', capacity = 1e-15)
        nodes_list_1 = []
        nodes_list_2 = []
        for i in range(10):
            for j in range(80):
                if f_1[i][j] !='0':
                    G.add_node(f'n_1[{i}][{j}]')
                    nodes_list_1.append((f'n_1[{i}][{j}]', j, int(f_1[i][j])/s1))
                    G.add_edge('s', f'n_1[{i}][{j}]', capacity = int(f_1[i][j])/s1)
        for i in range(10):
            for j in range(80):
                if f_2[i][j] !='0':
                    G.add_node(f'n_2[{i}][{j}]') 
                    nodes_list_2.append((f'n_2[{i}][{j}]', j, int(f_2[i][j])/s2))
                    G.add_edge(f'n_2[{i}][{j}]','t', capacity = int(f_2[i][j])/s2)
        for x in nodes_list_1: 
            for y in nodes_list_2:
                if x[1] < y[1]:                            
                    G.add_edge(x[0], y[0], weight = y[1] - x[1], capacity=min(x[2], y[2]))
                else:
                    G.add_edge(x[0], y[0], weight = 80 - x[1] + y[1], capacity=min(x[2], y[2]))
        flow = nx.min_cost_flow(G)
        emd_dist = 0
        for x in nodes_list_1: 
            for y in nodes_list_2:
                if x[1] < y[1]:                            
                    dist = y[1] - x[1]
                else:
                    dist = 80 - x[1] + y[1]
                emd_dist+= flow[x[0]][y[0]]*dist
   
    return float(emd_dist)
 

Secondly, we define the function which will return us the sorted order of the files. Note that we knoe that P1.txt is the first file in the list.

In [ ]:
def sort_files():          

    files = ['P1.txt', 'P2.txt', 'P3.txt', 'P4.txt', 'P5.txt', 'P6.txt', 'P7.txt', 'P8.txt', 'P9.txt', 'P10.txt', 'P11.txt', 'P12.txt','P13.txt', 'P14.txt', 'P15.txt']
    
    emd_distances = dict()
    for file in files: 
        emd_distances[file] = comp_dist('P1.txt',file)
    temp_files = []
    sorted_files = ['P1.txt']
    for elements in sorted(emd_distances.values()): 
        for names in emd_distances.keys():
            if emd_distances[names] == elements:
                if names not in sorted_files:
                    sorted_files.append(names)
                                                       
    return sorted_files